In [1]:
%xmode Verbose

# Imports
load('prismatic_envelope.sage')
load('precision.py')

Exception reporting mode: Verbose


### User-defined input

In [2]:
# The prime p
p=2

# The motivic weight i
i=5

# The power of the uniformizer n
n=2

# The residual degree f
# The present code only supports totally ramified extensions of Qp,
# i.e., where f=1
f=1

### Some precision calculations

In [3]:
# The calculated F-precision needed to compute at this weight
Fprec=n*i

# The target precision
target_precision=nygaard_exponent(p,i,n)

####################
# Precision losses #
####################

### From \delta
precision_loss_delta=math.floor(math.log(Fprec-1,p))

### From passing from OK to OK/pi^n
precision_loss_quotient=0
for q in range(p,i):
    precision_loss_quotient+=n*valuation(p,math.factorial(q))
    
### From lifting nabla to Nygaard
precision_loss_nygaard=n*math.floor(i*(i-1)/2)

### From computing can-phi on primitives
precision_loss_primitives=target_precision

### From renormalizing the Eisenstein polynomial
precision_loss_from_Eisenstein=1

total_precision=(target_precision+precision_loss_delta
                 +precision_loss_quotient
                 +precision_loss_nygaard
                 +precision_loss_primitives
                 +precision_loss_from_Eisenstein)

print("total p-adic precision is {}".format(total_precision))
print("Fprec is {}".format(Fprec))

# The coefficient ring W
if f==1:
    W=Zp(p,total_precision,type='capped-abs',print_mode='digits',show_prec=False)
else:
    W=Zq(p**f,total_precision,names='a',type='capped-abs',print_mode='series',show_prec=False)
    
# The Breuil-Kisin ring A
A.<z>=PowerSeriesRing(W,Fprec)

total p-adic precision is 43
Fprec is 10


### User-defined input: the Eisenstein polynomial

Note that in order for this to be created in a power series ring with the correct p-adic and F-adic precisions,
the elements p,i,n,f should be set above *before* defining the Eisenstein polynomial.

In [4]:
# The Eisenstein polynomial E
E=A(z+p)

### The main calculation

In [11]:
#%%capture
# Suppresses some Python warnings and SAGE variable injections

# The normalized Eisenstein polynomial
# The normalization is to bring the Eisenstein polynomial into the form E(0)=p
E=eisenstein_normalization(p,E)

# The syntomic matrices from new
%prun -s cumulative syn0,syn1,nablaN,nablaP=syntomic_matrices(p,i,n,E,total_precision,Fprec,debug=True)

Defining f0, f1, f2
fvars_phitilde:
[f0^2 + 10*f1, f1^2 + 10*f2, f2^2]


rels:
[(111111111111111111111111111111111111111111 + 111111111111111111111111111111111111111110*z + O(z^6))*f0^2 + (10 + z^2 + O(z^6))*f1, (11111111111111111111111111111111111111111 + 111111111111111111111111111111111111111110*z + O(z^2))*f0^4 + (100*z + O(z^2))*f0^2*f1 + (111111111111111111111111111111111111111101 + O(z^2))*f1^2 + (10 + O(z^2))*f2, f2^2]


fvars divided frobeniuses
[(1 + 11111111111111111111111111111111111111110*z + 100*z^2 + 111111111111111111111111111111111111000*z^3 + 10000*z^4 + 1111111111111111111111111111111100000*z^5)*f1, (10110110110110110110110110110110110111 + 111101011000110100001111101011000110100*z)*f2, 0]


can0:
[                                     100000                                           0                                           0                                           0                                           0                                           0          

<string>:1: DeprecationWarning: __getitem__ is changing to match the behavior of number fields. Please use expansion instead.
See http://trac.sagemath.org/14825 for details.


rels in prismatic_envelope_g:
[(11 + 10*z + z^2 + O(z^8))*g0^2 + ((1111111111111111111111111111111111111111110*z + 111111111111111111111111111111111111111111*z^2 + O(z^9))*d)*g0 + (10 + z^2 + O(z^8))*g1, (11111111111111111111111111111111111111101 + 111111111111111111111111111111111111111010*z + 11111111111111111111111111111111111111100*z^2 + 111111111111111111111111111111111111111110*z^3 + O(z^6))*g0^4 + ((110*z + 111*z^2 + 100*z^3 + z^4 + O(z^7))*d)*g0^3 + (111111111111111111111111111111111111111100*z + 11111111111111111111111111111111111111111*z^2 + 11111111111111111111111111111111111111110*z^3 + z^4 + O(z^6))*g0^2*g1 + ((111111111111111111111111111111111111111101*z^2 + 111111111111111111111111111111111111111110*z^3 + 11111111111111111111111111111111111111111*z^4 + O(z^8))*d^2)*g0^2 + ((100*z + 10*z^2 + 10*z^3 + z^4 + O(z^7))*d)*g0*g1 + (101 + 10*z^2 + 10*z^4 + O(z^6))*g1^2 + ((111111111111111111111111111111111111111110*z^2 + 111111111111111111111111111111111111111111*z^4 + O(z^8))*d

bk factor is
(110010101101101001110111000000111000*z + 1000101101001011010000101110111000000*z^2 + O(z^3))*g0*g1*g2 + (100100110110010011011001001101100101000*z + 10110100100010100011100111010100000*z^2 + 1110001000111011000001101010010110000*z^3 + 11110100001001010010001101110001010000*z^4 + 10010111110100011111010110011011101*z^5 + 11000111001110101001101111111011100*z^6 + O(z^7))*g0*g1 + (1010011001101111011011010100111111000*z + 10011111010101110001010111110010000*z^2 + 1000000010001100110101110011000110100*z^3 + 1000001000100111100010010110010111000*z^4 + O(z^5))*g0*g2 + (100100110110010011011001001101100101000 + 10100111011000000010010110011100110000*z + 10010111111011010000001001100100100*z^2 + 10010100000011100101111110100101000*z^3 + O(z^4))*g1*g2 + (1010101010101010101010101010101010101001*z + 11000111000111000111000111000111000111010*z^2 + 11001101001001010001000001100000100011*z^3 + 101000010111001001000110001010000110000*z^4 + 1110110001100001111010001101110100*z^5 + 10110

### Assembling the syntomic complex and computing its cohomology

In [6]:
# The K-groups (cohomology of the p-adic syntomic complex)
# New
coh_dict,final_precision=syntomic_cohomology(syn0,syn1,nablaN,nablaP)

print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-2)+' are {}'.format(coh_dict['h2'][1]))
print('Elementary divisors of K_{}(R;Z_p)'.format(2*i-1)+' are {}'.format(coh_dict['h1'][1]))
print('Target precision is {} and final precision is {}'.format(target_precision,final_precision))

Elementary divisors of K_8(R;Z_p) are []
Elementary divisors of K_9(R;Z_p) are [10, 10, 1000]
Target precision is 9 and final precision is 30
